# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, project_tests
%autoreload 1

2024-11-02 08:35:39.918852: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 08:35:39.918918: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 08:35:39.919019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 08:35:39.934183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Should return keras==2.14.0 and tf-keras==2.14.1
!pip freeze | grep keras

keras==2.14.0
tf-keras==2.14.1


In [3]:
# Should return tensorflow==2.14.0
!pip freeze | grep tensorflow

tensorflow==2.14.0
tensorflow-datasets==4.9.4
tensorflow-estimator==2.14.0
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.34.0
tensorflow-metadata==1.15.0


In [4]:
import collections

import helper
import numpy as np
import project_tests as tests

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [8]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [9]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    tokenized_x = tokenizer.texts_to_sequences(x)
    return tokenized_x, tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [10]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [11]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [12]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    inputs = Input(shape=input_shape[1:])
    rnn = SimpleRNN(64, return_sequences=True)(inputs)
    logits = Dense(french_vocab_size, activation='softmax')(rnn)
    model = Model(inputs=inputs, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(1e-3),
                  metrics=['accuracy'])
    return model


tests.test_simple_model(simple_model)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Epoch 1/20
108/108 [==============================] - 4s 26ms/step - loss: 3.2921 - accuracy: 0.4180 - val_loss: nan - val_accuracy: 0.4615
Epoch 2/20
108/108 [==============================] - 3s 23ms/step - loss: 2.3901 - accuracy: 0.4764 - val_loss: nan - val_accuracy: 0.5181
Epoch 3/20
108/108 [==============================] - 3s 24ms/step - loss: 2.1575 - accuracy: 0.5358 - val_loss: nan - val_accuracy: 0.5469
Epoch 4/20
108/108 [==============================] - 3s 23ms/step - loss: 1.9631 - accuracy: 0.5538 - val_loss: nan - val_accuracy: 0.5648
Epoch 5/20
108/108 [==============================] - 3s 23ms/step - loss: 1.8078 - accuracy: 0.5701 - val_loss: nan - val_accuracy: 0.5809
Epoch 6/20
108/108 [==============================] - 3s 23ms/step - loss: 1.7025 - accuracy: 0.5859 - val_loss: nan - val_accuracy: 0.5911
Epoch 7/20
108/108 [==============================] - 3s 23ms/step - loss: 1.6337 - accuracy: 0.5923 - val_loss: nan - val_accuracy: 0.5972
Epoch 8/20
108/108 [

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Define the input layer
    inputs = Input(shape=input_shape[1:])

    # Add Embedding layer
    embedding = Embedding(input_dim=english_vocab_size, output_dim=128)(inputs)

    # Add RNN layer
    rnn = SimpleRNN(64, return_sequences=True)(embedding)

    # Add Dense layer
    logits = Dense(french_vocab_size, activation='softmax')(rnn)

    # Define the model
    model = Model(inputs=inputs, outputs=logits)

    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(1e-3),
                  metrics=['accuracy'])
    return model


# Test the embed_model function
tests.test_embed_model(embed_model)

# Reshape the input to work with the embedding RNN model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Epoch 1/10
108/108 [==============================] - 8s 53ms/step - loss: 3.4629 - accuracy: 0.4134 - val_loss: nan - val_accuracy: 0.4923
Epoch 2/10
108/108 [==============================] - 4s 37ms/step - loss: 2.0839 - accuracy: 0.5657 - val_loss: nan - val_accuracy: 0.6245
Epoch 3/10
108/108 [==============================] - 3s 31ms/step - loss: 1.4015 - accuracy: 0.6763 - val_loss: nan - val_accuracy: 0.7236
Epoch 4/10
108/108 [==============================] - 3s 32ms/step - loss: 1.0410 - accuracy: 0.7513 - val_loss: nan - val_accuracy: 0.7767
Epoch 5/10
108/108 [==============================] - 3s 32ms/step - loss: 0.8451 - accuracy: 0.7929 - val_loss: nan - val_accuracy: 0.8073
Epoch 6/10
108/108 [==============================] - 3s 32ms/step - loss: 0.7235 - accuracy: 0.8181 - val_loss: nan - val_accuracy: 0.8282
Epoch 7/10
108/108 [==============================] - 3s 32ms/step - loss: 0.6420 - accuracy: 0.8351 - val_loss: nan - val_accuracy: 0.8412
Epoch 8/10
108/108 [

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, SimpleRNN, Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Define the input layer with shape including the features as singleton dimension
    inputs = Input(shape=(input_shape[1], input_shape[2]))

    # Flatten the input to remove the singleton dimension
    reshape = Reshape((input_shape[1],))(inputs)

    # Add Embedding layer to input
    embedding = Embedding(input_dim=english_vocab_size, output_dim=128, input_length=input_shape[1])(reshape)

    # Add Bidirectional RNN layer
    bidirectional_rnn = Bidirectional(SimpleRNN(64, return_sequences=True))(embedding)

    # Add Dense layer
    logits = Dense(french_vocab_size, activation='softmax')(bidirectional_rnn)

    # Define the model
    model = Model(inputs=inputs, outputs=logits)

    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(1e-3),
                  metrics=['accuracy'])
    return model


# Test the bd_model function
tests.test_bd_model(bd_model)

# Reshape the input to work with the bidirectional RNN model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Add singleton dimension to match input shape (batch_size, timesteps, features)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# Train the neural network
bidirectional_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bidirectional_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidirectional_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Epoch 1/10
108/108 [==============================] - 11s 75ms/step - loss: 2.9373 - accuracy: 0.4581 - val_loss: nan - val_accuracy: 0.5368
Epoch 2/10
108/108 [==============================] - 6s 57ms/step - loss: 1.6115 - accuracy: 0.6229 - val_loss: nan - val_accuracy: 0.6914
Epoch 3/10
108/108 [==============================] - 6s 57ms/step - loss: 1.0849 - accuracy: 0.7383 - val_loss: nan - val_accuracy: 0.7736
Epoch 4/10
108/108 [==============================] - 6s 56ms/step - loss: 0.8067 - accuracy: 0.7936 - val_loss: nan - val_accuracy: 0.8124
Epoch 5/10
108/108 [==============================] - 6s 56ms/step - loss: 0.6525 - accuracy: 0.8274 - val_loss: nan - val_accuracy: 0.8414
Epoch 6/10
108/108 [==============================] - 6s 55ms/step - loss: 0.5519 - accuracy: 0.8521 - val_loss: nan - val_accuracy: 0.8628
Epoch 7/10
108/108 [==============================] - 6s 54ms/step - loss: 0.4806 - accuracy: 0.8711 - val_loss: nan - val_accuracy: 0.8779
Epoch 8/10
108/108 

### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [45]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Dropout, RepeatVector
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.001
    lstm_dim = 512  # Increase number of neurons in LSTM
    dense_units = 1024

    # Encoder
    encoder_inputs = Input(shape=input_shape[1:])
    encoder_lstm = LSTM(lstm_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = RepeatVector(output_sequence_length)(state_h)  # Use state_h for RepeatVector
    decoder_lstm = LSTM(lstm_dim, return_sequences=True)(decoder_inputs, initial_state=encoder_states)
    decoder_dense_1 = TimeDistributed(Dense(dense_units, activation='relu'))(decoder_lstm)
    dropout_out = Dropout(0.5)(decoder_dense_1)
    decoder_outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dropout_out)

    # Define the model
    model = Model(inputs=encoder_inputs, outputs=decoder_outputs)

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])

    return model


# Test the encdec_model function
tests.test_encdec_model(encdec_model)

# Prepare the input data
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, tmp_x.shape[1], 1))

# Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape,  # Adjust the shape of input
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)

encdec_rnn_model.summary()

encdec_model_chart = encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30,
                                          validation_split=0.2)

print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_49 (InputLayer)       [(None, 21, 1)]              0         []                            
                                                                                                  
 lstm_40 (LSTM)              [(None, 512),                1052672   ['input_49[0][0]']            
                              (None, 512),                                                        
                              (None, 512)]                                                        
                                                                                                  
 repeat_vector_6 (RepeatVec  (None, 21, 512)              0         ['lstm_40[0][1]']             
 tor)                                                                                      

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [47]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional, RepeatVector
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy


def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.001
    lstm_dim = 512  # Increase the number of neurons in LSTM
    embedding_dim = 128
    dense_units = 1024

    # Encoder
    encoder_inputs = Input(shape=input_shape[1:])
    encoder_embedding = Embedding(input_dim=english_vocab_size, output_dim=embedding_dim, input_length=input_shape[1])(
        encoder_inputs)
    encoder_bi_lstm = Bidirectional(LSTM(lstm_dim, return_sequences=False, return_state=True))
    _, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_embedding)

    # Compute the combined states
    state_h = forward_h + backward_h
    state_c = forward_c + backward_c
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = RepeatVector(output_sequence_length)(state_h)
    decoder_lstm = LSTM(lstm_dim, return_sequences=True)
    decoder_lstm_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense_1 = TimeDistributed(Dense(dense_units, activation='relu'))(decoder_lstm_outputs)
    dropout_out = Dropout(0.5)(decoder_dense_1)
    decoder_outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dropout_out)

    # Define the model
    model = Model(inputs=encoder_inputs, outputs=decoder_outputs)

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])

    return model


# Test the model_final function
tests.test_model_final(model_final)

print('Final Model Loaded')

# Prepare the input data
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, tmp_x.shape[1]))

# Train and Print prediction(s)
final_rnn_model = model_final(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)

final_rnn_model.summary()

# Train the model
final_model_chart = final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=30,
                                        validation_split=0.2)

# Print predictions
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Final Model Loaded
Model: "model_31"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_53 (InputLayer)       [(None, 21)]                 0         []                            
                                                                                                  
 embedding_33 (Embedding)    (None, 21, 128)              25600     ['input_53[0][0]']            
                                                                                                  
 bidirectional_10 (Bidirect  [(None, 1024),               2625536   ['embedding_33[0][0]']        
 ional)                       (None, 512),                                                        
                              (None, 512),                                                        
                              (None, 512),                              

1/1 [==============================] - 1s 1s/step
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Prediction (IMPLEMENTATION)

In [49]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using model_final
    model = model_final(
        x.shape,
        y.shape[1],
        len(x_tk.word_index) + 1,
        len(y_tk.word_index) + 1
    )

    # Compile and train the model
    model.fit(x, y, batch_size=1024, epochs=30, validation_split=0.2)

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


# Preparing the input data
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, tmp_x.shape[1]))

final_predictions(tmp_x, preproc_french_sentences, english_tokenizer, french_tokenizer)


Epoch 1/30
108/108 [==============================] - 36s 271ms/step - loss: 2.6314 - accuracy: 0.4736 - val_loss: 1.7152 - val_accuracy: 0.5680
Epoch 2/30
108/108 [==============================] - 28s 258ms/step - loss: 1.5137 - accuracy: 0.5987 - val_loss: 1.2734 - val_accuracy: 0.6453
Epoch 3/30
108/108 [==============================] - 27s 254ms/step - loss: 1.2325 - accuracy: 0.6581 - val_loss: 1.0331 - val_accuracy: 0.7082
Epoch 4/30
108/108 [==============================] - 28s 256ms/step - loss: 1.0232 - accuracy: 0.7074 - val_loss: 0.9024 - val_accuracy: 0.7337
Epoch 5/30
108/108 [==============================] - 28s 257ms/step - loss: 0.8950 - accuracy: 0.7333 - val_loss: 0.7839 - val_accuracy: 0.7602
Epoch 6/30
108/108 [==============================] - 28s 255ms/step - loss: 0.7644 - accuracy: 0.7640 - val_loss: 0.6724 - val_accuracy: 0.7889
Epoch 7/30
108/108 [==============================] - 28s 256ms/step - loss: 0.6581 - accuracy: 0.7932 - val_loss: 0.5610 - val_ac

## Submission
When you're ready to submit, complete the following steps:
1. Review the rubric to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [53]:
# Save before you run this cell!
!jupyter nbconvert *.ipynb --to html

[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html
[NbConvertApp] Writing 330691 bytes to machine_translation-zh.html
[NbConvertApp] Converting notebook machine_translation.ipynb to html
[NbConvertApp] Writing 404172 bytes to machine_translation.html


## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?